In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, Flatten
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.optimizers import SGD
import keras_metrics
from sklearn import metrics
from sklearn.utils import shuffle
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from sklearn.utils import shuffle, class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

Using TensorFlow backend.


In [2]:
labels = pd.read_csv('labels.csv',header=0)
print(list(labels.columns.values))
print("\n")
labels= labels[pd.notnull(labels['gene'])]
# df.value_counts()

    
    
data=pd.read_csv('data.csv',header=0)
print(list(data.columns.values))
data = data[pd.notnull(data['ORF'])]

['gene', 'TCA', 'Resp', 'Ribo', 'Proteas', 'Hist', 'HTH']


['ORF', 'NAME', 'alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'alpha 63', 'alpha 70', 'alpha 77', 'alpha 84', 'alpha 91', 'alpha 98', 'alpha 105', 'alpha 112', 'alpha 119', 'Elu 0', 'Elu 30', 'Elu 60', 'Elu 90', 'Elu 120', 'Elu 150', 'Elu 180', 'Elu 210', 'Elu 240', 'Elu 270', 'Elu 300', 'Elu 330', 'Elu 360', 'Elu 390', 'cdc15 10', 'cdc15 30', 'cdc15 50', 'cdc15 70', 'cdc15 90', 'cdc15 110', 'cdc15 130', 'cdc15 150', 'cdc15 170', 'cdc15 190', 'cdc15 210', 'cdc15 230', 'cdc15 250', 'cdc15 270', 'cdc15 290', 'spo 0', 'spo 2', 'spo 5', 'spo 7', 'spo 9', 'spo 11', 'spo5 2', 'spo5 7', 'spo5 11', 'spo- early', 'spo- mid', 'heat 0', 'heat 10', 'heat 20', 'heat 40', 'heat 80', 'heat 160', 'dtt 15', 'dtt 30', 'dtt 60', 'dtt 120', 'cold 0', 'cold 20', 'cold 40', 'cold 160', 'diau a', 'diau b', 'diau c', 'diau d', 'diau e', 'diau f', 'diau g']


In [3]:
data.head(5)

,ORF,NAME,alpha 0,alpha 7,alpha 14,alpha 21,alpha 28,alpha 35,alpha 42,alpha 49,...,cold 20,cold 40,cold 160,diau a,diau b,diau c,diau d,diau e,diau f,diau g
0,YBR166C,TYR1 TYROSINE BIOSYNTHESIS PREPHENATE DEH...,0.33,-0.17,0.04,-0.07,-0.09,-0.12,-0.03,-0.20,...,0.08,0.39,-0.17,0.23,0.20,0.20,-0.17,-0.69,0.14,-0.27
1,YOR357C,GRD19 SECRETION GOLGI PROTEIN ...,-0.64,-0.38,-0.32,-0.29,-0.22,-0.01,-0.32,-0.27,...,0.36,-0.20,-0.34,-0.14,-0.09,0.06,-0.17,0.04,-0.97,-1.79
2,YLR292C,SEC72 SECRETION ER PROTEIN TRA...,-0.23,0.19,-0.36,0.14,-0.40,0.16,-0.09,-0.12,...,0.21,-0.29,-0.36,-0.10,-0.29,-0.18,-0.34,-0.47,-0.43,-1.06
3,YGL112C,TAF60 TRANSCRIPTION TFIID 60 KD SU...,-0.69,-0.89,-0.74,-0.56,-0.64,-0.18,-0.42,-0.34,...,-0.01,-0.29,0.31,0.21,0.07,0.18,-0.14,-0.20,-0.43,-1.51
4,YIL118W,RHO3 CYTOSKELETON GTP-BINDING PR...,0.04,0.01,-0.81,NaN,-0.30,0.49,0.08,0.19,...,0.28,-0.17,0.26,0.07,-0.17,NaN,-0.10,-0.23,-0.51,-1.40


In [4]:
labels.head(5)

,gene,TCA,Resp,Ribo,Proteas,Hist,HTH
1,YAL001C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,YAL002W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,YAL003W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,YAL005C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,YAL010C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [5]:
X=np.loadtxt('newdata.txt')
Y=np.loadtxt('newlabels.txt')
Y=Y.astype(int)

In [6]:
print(X.shape)
print(Y.shape)
X,Y= shuffle(X,Y)

(2464, 79)
(2464,)


In [8]:
n_values = np.max(Y) + 1
newY=np.eye(n_values)[Y] #one hot encoding for softmax
print(newY.shape)

(2464, 7)


In [9]:
n=int(X.shape[0]*0.80)
length=X.shape[0]
X_train=X[0:n,:]
X_test=X[n:,:]

Y_train=Y[0:n]
Y_test=Y[n:]

newY_train=newY[0:n]
newY_test=newY[n:]

In [10]:
print(X_train.shape)
print(X_test.shape)

(1971, 79)
(493, 79)


In [11]:
print(Y_train.shape)
print(Y_test.shape)

print("One hot encoded Y shape")
print(newY_train.shape)
print(newY_test.shape)

(1971,)
(493,)
One hot encoded Y shape
(1971, 7)
(493, 7)


In [12]:
#using the sklearn class weights to find find the weights of the balanced class
Y_train=Y_train.reshape((Y_train.shape[0],))
print(Y_train.shape)
class_weights_values= class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
Y_train=Y_train.reshape(-1, 1)

(1971,)


In [61]:
seed = 7
np.random.seed(seed)
kfold=KFold(n_splits=4, shuffle=True, random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
for train, test in kfold.split(X_train, newY_train,Y_train):
    # create model
    model = Sequential()
    model.add(Dense(150, input_dim=79, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(45, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
    
    model.fit(X_train[train], newY_train[train], epochs=10, batch_size=10, verbose=1,class_weight=class_weights_values)
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1)
    
    labels=[0,1,2,3,4,5,6]
    #f1 values
    f1_value=metrics.f1_score(predict_class,Y_train[test],labels,average='macro')
    f1_class_value=metrics.f1_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(predict_class,Y_train[test],labels,average='macro')
    precision_class_value=metrics.precision_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,Y_train[test],labels,average='macro')
    recall_class_value=metrics.recall_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    scores = model.evaluate(X_train[test], newY_train[test], verbose=1)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

Epoch 1/10
1478/1478 [==============================] - 7s 5ms/step - loss: 0.5011 - acc: 0.8829
Epoch 2/10
1478/1478 [==============================] - 1s 490us/step - loss: 0.2608 - acc: 0.9344
Epoch 3/10
1478/1478 [==============================] - 1s 491us/step - loss: 0.2224 - acc: 0.9459
Epoch 4/10
1478/1478 [==============================] - 1s 487us/step - loss: 0.1897 - acc: 0.9587
Epoch 5/10
1478/1478 [==============================] - 1s 489us/step - loss: 0.1731 - acc: 0.9574
Epoch 6/10
1478/1478 [==============================] - 1s 489us/step - loss: 0.1401 - acc: 0.9614
Epoch 7/10
1478/1478 [==============================] - 1s 492us/step - loss: 0.1107 - acc: 0.9696
Epoch 8/10
1478/1478 [==============================] - 1s 489us/step - loss: 0.1374 - acc: 0.9614
Epoch 9/10
1478/1478 [==============================] - 1s 473us/step - loss: 0.1116 - acc: 0.9614
Epoch 10/10
1478/1478 [==============================] - 1s 407us/step - loss: 0.1206 - acc: 0.9628


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Epoch 1/10
1478/1478 [==============================] - 7s 5ms/step - loss: 0.5027 - acc: 0.8999
Epoch 2/10
1478/1478 [==============================] - 1s 409us/step - loss: 0.2271 - acc: 0.9452
Epoch 3/10
1478/1478 [==============================] - 1s 427us/step - loss: 0.1739 - acc: 0.9553
Epoch 4/10
1478/1478 [==============================] - 1s 452us/step - loss: 0.1769 - acc: 0.9641
Epoch 5/10
1478/1478 [==============================] - 1s 404us/step - loss: 0.1336 - acc: 0.9635
Epoch 6/10
1478/1478 [==============================] - 1s 433us/step - loss: 0.1365 - acc: 0.9689
Epoch 7/10
1478/1478 [==============================] - 1s 394us/step - loss: 0.1127 - acc: 0.9729
Epoch 8/10
1478/1478 [==============================] - 1s 398us/step - loss: 0.1220 - acc: 0.9648
Epoch 9/10
1478/1478 [==============================] - 1s 409us/step - loss: 0.0963 - acc: 0.9743
Epoch 10/10
1478/1478 [==============================] - 1s 449us/step - loss: 0.0894 - acc: 0.9716
Epoch 1/10


In [62]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.971078843648477
F1 is :0.5926644299516923
Average F1 for all the classes is:[0.98477663 0.23809524 0.57386364 0.96193759 0.77331125 0.61666667
 0.        ]


In [99]:
#Based on the other models, we will compare the score against the other models
#and then see how well the model will generalise based on the test scores.

In [21]:
seed =7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #automatically balance all the classes of the training data
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train[train], Y_train[train]) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    
    """
    pca= PCA(50)
    X_under= pca.fit_transform(X_under)
    newX_test=pca.transform(X_train[test])
    """
    
    model = Sequential()
    model.add(Dense(175, input_dim=79, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=32, verbose=1)
    
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(X_train[test], newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9331/9331 [==============================] - 1s 86us/step - loss: 0.3262 - acc: 0.8938 - precision: 0.8514 - recall: 0.5844
Epoch 2/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0788 - acc: 0.9767 - precision: 0.9430 - recall: 0.9062
Epoch 3/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0460 - acc: 0.9853 - precision: 0.9630 - recall: 0.9377
Epoch 4/10
9331/9331 [==============================] - 0s 52us/step - loss: 0.0331 - acc: 0.9909 - precision: 0.9800 - recall: 0.9557
Epoch 5/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0326 - acc: 0.9917 - precision: 0.9846 - recall: 0.9617
Epoch 6/10
9331/9331 [==============================] - 1s 60us/step - loss: 0.0289 - acc: 0.9915 - precision: 0.9758 - recall: 0.9662
Epoch 7/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0307 - acc: 0.9915 - precision: 0.9810 - recall: 0.9670
Epoch 8/10
9331/9331 [==============================] -

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9331/9331 [==============================] - 1s 92us/step - loss: 0.2936 - acc: 0.9013 - precision: 0.8852 - recall: 0.6189
Epoch 2/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0675 - acc: 0.9787 - precision: 0.9563 - recall: 0.9025
Epoch 3/10
9331/9331 [==============================] - 0s 52us/step - loss: 0.0437 - acc: 0.9851 - precision: 0.9630 - recall: 0.9377
Epoch 4/10
9331/9331 [==============================] - 1s 66us/step - loss: 0.0328 - acc: 0.9922 - precision: 0.9817 - recall: 0.9655
Epoch 5/10
9331/9331 [==============================] - 1s 55us/step - loss: 0.0214 - acc: 0.9938 - precision: 0.9863 - recall: 0.9692
Epoch 6/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0186 - acc: 0.9947 - precision: 0.9909 - recall: 0.9760
Epoch 7/10
9331/9331 [==============================] - 0s 53us/step - loss: 0.0219 - acc: 0.9938 - precision: 0.9856 - recall: 0.9745
Epoch 8/10
9331/9331 [==============================] -

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9338/9338 [==============================] - 1s 88us/step - loss: 0.3029 - acc: 0.8999 - precision: 0.8469 - recall: 0.6222
Epoch 2/10
9338/9338 [==============================] - 0s 52us/step - loss: 0.0788 - acc: 0.9780 - precision: 0.9465 - recall: 0.9018
Epoch 3/10
9338/9338 [==============================] - 0s 53us/step - loss: 0.0487 - acc: 0.9860 - precision: 0.9683 - recall: 0.9378
Epoch 4/10
9338/9338 [==============================] - 1s 60us/step - loss: 0.0376 - acc: 0.9898 - precision: 0.9763 - recall: 0.9558
Epoch 5/10
9338/9338 [==============================] - 1s 54us/step - loss: 0.0309 - acc: 0.9913 - precision: 0.9779 - recall: 0.9640
Epoch 6/10
9338/9338 [==============================] - 1s 58us/step - loss: 0.0210 - acc: 0.9936 - precision: 0.9856 - recall: 0.9723
Epoch 7/10
9338/9338 [==============================] - 1s 56us/step - loss: 0.0250 - acc: 0.9928 - precision: 0.9811 - recall: 0.9715
Epoch 8/10
9338/9338 [==============================] -

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
9338/9338 [==============================] - 1s 99us/step - loss: 0.3357 - acc: 0.8880 - precision: 0.8702 - recall: 0.6229
Epoch 2/10
9338/9338 [==============================] - 1s 76us/step - loss: 0.0835 - acc: 0.9754 - precision: 0.9404 - recall: 0.8868
Epoch 3/10
9338/9338 [==============================] - 1s 72us/step - loss: 0.0537 - acc: 0.9850 - precision: 0.9658 - recall: 0.9310
Epoch 4/10
9338/9338 [==============================] - 1s 63us/step - loss: 0.0480 - acc: 0.9871 - precision: 0.9699 - recall: 0.9415
Epoch 5/10
9338/9338 [==============================] - 1s 72us/step - loss: 0.0352 - acc: 0.9908 - precision: 0.9787 - recall: 0.9625
Epoch 6/10
9338/9338 [==============================] - 1s 64us/step - loss: 0.0284 - acc: 0.9921 - precision: 0.9869 - recall: 0.9595
Epoch 7/10
9338/9338 [==============================] - 1s 62us/step - loss: 0.0247 - acc: 0.9936 - precision: 0.9818 - recall: 0.9730
Epoch 8/10
9338/9338 [==============================] -

In [22]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9649976491039663
F1 is :0.68851661486299
Average F1 for all the classes is:[0.98143161 0.44588745 0.73928571 0.94641947 0.85540158 0.76785714
 0.08333333]


In [102]:
seed = 7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #pca for all features
    alphaX= X_train[train][:,0:18]
    eluX= X_train[train][:,18:32]
    edcX= X_train[train][:,32:47]
    spotX= X_train[train][:,47:56]
    spoX= X_train[train][:,56:58]
    heatX= X_train[train][:,58:64]
    dttX= X_train[train][:,64:68]
    coldX= X_train[train][:,68:72]
    diauX= X_train[train][:,72:79]

    alphaX_test= X_train[test][:,0:18]
    eluX_test= X_train[test][:,18:32]
    edcX_test= X_train[test][:,32:47]
    spotX_test= X_train[test][:,47:56]
    spoX_test= X_train[test][:,56:58]
    heatX_test= X_train[test][:,58:64]
    dttX_test= X_train[test][:,64:68]
    coldX_test= X_train[test][:,68:72]
    diauX_test= X_train[test][:,72:79]
    
    pca= PCA(1)
    
    alphaX= pca.fit_transform(alphaX)
    alphaX_test=pca.transform(alphaX_test)
    
    eluX= pca.fit_transform(eluX)
    eluX_test=pca.transform(eluX_test)
    
    edcX= pca.fit_transform(edcX)
    edcX_test=pca.transform(edcX_test)
    
    spotX= pca.fit_transform(spotX)
    spotX_test=pca.transform(spotX_test)
    
    spoX= pca.fit_transform(spoX)
    spoX_test=pca.transform(spoX_test)
    
    heatX= pca.fit_transform(heatX)
    heatX_test=pca.transform(heatX_test)
    
    dttX= pca.fit_transform(dttX)
    dttX_test=pca.transform(dttX_test)
    
    coldX= pca.fit_transform(coldX)
    coldX_test=pca.transform(coldX_test)
    
    diauX= pca.fit_transform(diauX)
    diauX_test=pca.transform(diauX_test)
    
    x_train=np.concatenate([alphaX,eluX,edcX,spotX,spoX,heatX,dttX,coldX,diauX],axis=1)
    x_test=np.concatenate([alphaX_test,eluX_test,edcX_test,spotX_test,spoX_test,heatX_test,dttX_test,coldX_test,diauX_test],axis=1)
    print(x_train.shape)
    print(x_test.shape)
    
    #over and undersampling-balancing all the classes
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(x_train, Y_train[train])
    X_under,y_under= shuffle(X_under,y_under)
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    
    """
    pca= PCA(50)
    X_under= pca.fit_transform(X_under)
    newX_test=pca.transform(X_train[test])
    """
    
    model = Sequential()
    model.add(Dense(175, input_dim=9, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=32, verbose=1)
    
    y_pred=model.predict(x_test)
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(x_test, newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

(1475, 9)
(496, 9)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/5
9373/9373 [==============================] - 4s 441us/step - loss: 0.5758 - acc: 0.7917 - precision: 0.7476 - recall: 0.3495
Epoch 2/5
9373/9373 [==============================] - 2s 211us/step - loss: 0.2650 - acc: 0.9108 - precision: 0.8436 - recall: 0.6326
Epoch 3/5
9373/9373 [==============================] - 2s 210us/step - loss: 0.1875 - acc: 0.9425 - precision: 0.8849 - recall: 0.7461
Epoch 4/5
9373/9373 [==============================] - 2s 211us/step - loss: 0.1552 - acc: 0.9504 - precision: 0.9062 - recall: 0.7797
Epoch 5/5
9373/9373 [==============================] - 2s 217us/step - loss: 0.1316 - acc: 0.9612 - precision: 0.9163 - recall: 0.8260
(1476, 9)
(495, 9)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/5
9373/9373 [==============================] - 4s 390us/step - loss: 0.5725 - acc: 0.8049 - precision: 0.7086 - recall: 0.3032
Epoch 2/5
9373/9373 [==============================] - 2s 176us/step - loss: 0.2504 - acc: 0.9165 - precision: 0.8137 - recall: 0.6102
Epoch 3/5
9373/9373 [==============================] - 2s 199us/step - loss: 0.1854 - acc: 0.9398 - precision: 0.8641 - recall: 0.7072
Epoch 4/5
9373/9373 [==============================] - 2s 197us/step - loss: 0.1546 - acc: 0.9515 - precision: 0.8943 - recall: 0.7580
Epoch 5/5
9373/9373 [==============================] - 2s 197us/step - loss: 0.1345 - acc: 0.9584 - precision: 0.9130 - recall: 0.7834
(1480, 9)
(491, 9)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/5
9380/9380 [==============================] - 4s 475us/step - loss: 0.5948 - acc: 0.7923 - precision: 0.7246 - recall: 0.3142
Epoch 2/5
9380/9380 [==============================] - 2s 214us/step - loss: 0.2647 - acc: 0.9098 - precision: 0.8224 - recall: 0.6015
Epoch 3/5
9380/9380 [==============================] - 2s 248us/step - loss: 0.2019 - acc: 0.9364 - precision: 0.8691 - recall: 0.6836
Epoch 4/5
9380/9380 [==============================] - 2s 228us/step - loss: 0.1668 - acc: 0.9477 - precision: 0.8897 - recall: 0.7463
Epoch 5/5
9380/9380 [==============================] - 2s 239us/step - loss: 0.1449 - acc: 0.9549 - precision: 0.9083 - recall: 0.7836
(1482, 9)
(489, 9)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/5
9380/9380 [==============================] - 4s 438us/step - loss: 0.6050 - acc: 0.7814 - precision: 0.7465 - recall: 0.3187
Epoch 2/5
9380/9380 [==============================] - 2s 213us/step - loss: 0.2914 - acc: 0.9027 - precision: 0.8107 - recall: 0.5881
Epoch 3/5
9380/9380 [==============================] - 2s 229us/step - loss: 0.2160 - acc: 0.9308 - precision: 0.8547 - recall: 0.6761
Epoch 4/5
9380/9380 [==============================] - 2s 232us/step - loss: 0.1687 - acc: 0.9480 - precision: 0.9013 - recall: 0.7425
Epoch 5/5
9380/9380 [==============================] - 2s 219us/step - loss: 0.1475 - acc: 0.9520 - precision: 0.9101 - recall: 0.7709


In [103]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.8386203736297773
F1 is :0.44108188516147284
Average F1 for all the classes is:[0.90662082 0.23121364 0.32869756 0.76065808 0.4583194  0.3177521
 0.08431159]
